In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import json
import geopandas as gpd
from copy import deepcopy
import shapely
from shapely.geometry import Point,Polygon,LineString
import pyproj

### map of the 50 metro areas

In [82]:
# shape file for US metro areas
metromap = gpd.read_file("../data/shapes/tl_2019_us_cbsa/tl_2019_us_cbsa.shp")

In [83]:
# manipulations
metromap.set_geometry("geometry",inplace=True)

metromap["GEOID"] = metromap["GEOID"].astype(int)

In [84]:
# cbsa info table
cbsa_info = pd.read_csv("../data/cbsacode_shortname_tracts.csv", sep=";", index_col=0)

In [85]:
# manipulations
cbsa_info = cbsa_info.loc[:,["cbsacode", "short_name"]].drop_duplicates()
cbsa_info["selected"] = 1
cbsa_info["color"] = "darkblue"

In [86]:
# merge for plot
metromap = pd.merge(metromap, cbsa_info, left_on="GEOID", right_on="cbsacode", how="left")

In [87]:
# correction on the final dataframe
metromap["selected"] = metromap["selected"].fillna(0)
metromap["color"] = metromap["color"].fillna("lightgray")

In [88]:
# shape file for US states
usmap = gpd.read_file("../data/shapes/tl_2017_us_state/tl_2017_us_state.shp")

# manipulations
usmap.set_geometry("geometry",inplace=True)

### user numbers

In [89]:
# data IN -- edges are present both ways
geo_edgelist = pd.read_csv("../data/geo_edgelist_top50.csv.gz")

# degree
degree_tab = geo_edgelist.groupby("user_id1")["user_id2"].count().reset_index()
degree_tab.columns = ["user_id", "degree"]

# remove users with less than 10 ties (!!!) AND replace NANs
degree_tab = degree_tab[degree_tab["degree"]>=10] 
degree_tab = degree_tab.fillna(0)

# user and cbsa
users = geo_edgelist[["cbsacode", "user_id1"]].drop_duplicates()

# merge
degree_tab = pd.merge(degree_tab, users, left_on="user_id", right_on="user_id1", how="left")

# user counnt per cbsa
users_cbsa = degree_tab.groupby("cbsacode")["user_id"].count().reset_index()

In [90]:
# METRODATA

metrodata = pd.read_csv('../data/cbsa-est2018-alldata.csv', encoding = "ISO-8859-1")
metrodata = metrodata.drop_duplicates(subset=["CBSA"])

# keep relevant cbsacodes ONLY - sort by 2010 population
metrodata = metrodata.set_index("CBSA").loc[users_cbsa["cbsacode"].unique()].reset_index().sort_values(by="CENSUS2010POP",ascending=False)
metrodata = metrodata.reset_index().reset_index()[["level_0","CBSA","NAME","CENSUS2010POP"]]
metrodata.rename({"level_0":"rank","CBSA":"cbsacode","NAME":"name","CENSUS2010POP":"population"},axis=1,inplace=True)

# create short name for CBSAs
metrodata["short_name"] = metrodata["name"].map(lambda s: s.split("-")[0].split(",")[0])

metrodata.loc[metrodata.short_name == "Louisville/Jefferson County", "short_name"] = "Louisville"

# merge user counts
metrodata = pd.merge(users_cbsa, metrodata, on="cbsacode")
metrodata = metrodata.sort_values(by=['rank'])

In [ ]:
### combination

# general
import matplotlib.patches as mpatches
plt.rcParams['font.size']=16
plt.rc('legend',fontsize=22)
fig, ax = plt.subplots(2,1, figsize = (15,15), constrained_layout=True)


# MAP
usmap.plot(ax=ax[0], color="white", edgecolor='dimgray')
metromap.plot(ax=ax[0], column="selected", color=metromap["color"], edgecolor='dimgray')
ax[0].set_xlim([-127,-66])
ax[0].set_ylim([24,50])
ax[0].axis('off')

top50metro = mpatches.Patch(color='darkblue', label='Top 50 metro areas')
allmetro = mpatches.Patch(color='lightgray', label='Metro areas')
ax[0].legend(handles=[top50metro, allmetro], loc='lower left')


# distribution
ax[1].bar(metrodata["short_name"], metrodata["user_id"], color="darkblue")
ax[1].set_xticklabels(labels="", rotation=90)
ax[1].grid()
# ax[1].semilogy()
ax[1].set_xticklabels(labels=metrodata['short_name'], rotation=90)
ax[1].set_ylabel("Observed users", size=22)

plt.show()
# plt.savefig('../fig/map_users_top50.png', dpi=300, bbox_inches='tight')